# feature vggish

기존 코드 

In [1]:
import pickle as pk
import glob
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from tools.utils import load_train_valid, get_class_distribution,get_logger, multi_acc, load_test


import gc
#from torch.utils.tensorboard import SummaryWriter
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
from models import *
from get_feature import *
import pickle as pk

In [2]:
data_folder = '/Data2/hmd/data_split/murmur/train'

In [3]:
model_folder = 'model'

In [4]:
patient_files = find_patient_files(data_folder)

In [5]:
patient_files

['/Data2/hmd/data_split/murmur/train/2530.txt',
 '/Data2/hmd/data_split/murmur/train/9979.txt',
 '/Data2/hmd/data_split/murmur/train/9983.txt',
 '/Data2/hmd/data_split/murmur/train/13918.txt',
 '/Data2/hmd/data_split/murmur/train/14241.txt',
 '/Data2/hmd/data_split/murmur/train/14998.txt',
 '/Data2/hmd/data_split/murmur/train/23625.txt',
 '/Data2/hmd/data_split/murmur/train/24160.txt',
 '/Data2/hmd/data_split/murmur/train/29045.txt',
 '/Data2/hmd/data_split/murmur/train/29378.txt',
 '/Data2/hmd/data_split/murmur/train/31737.txt',
 '/Data2/hmd/data_split/murmur/train/36327.txt',
 '/Data2/hmd/data_split/murmur/train/39043.txt',
 '/Data2/hmd/data_split/murmur/train/39403.txt',
 '/Data2/hmd/data_split/murmur/train/39456.txt',
 '/Data2/hmd/data_split/murmur/train/40058.txt',
 '/Data2/hmd/data_split/murmur/train/40840.txt',
 '/Data2/hmd/data_split/murmur/train/43852.txt',
 '/Data2/hmd/data_split/murmur/train/44514.txt',
 '/Data2/hmd/data_split/murmur/train/45843.txt',
 '/Data2/hmd/data_split

In [8]:
num_patient_files = len(patient_files)

In [9]:
num_patient_files #751명의 환자 데이터

751

In [10]:
features, labels = get_features(data_folder, patient_files)

In [12]:
features['mel1'].shape

(2532, 64, 313, 1)

In [6]:
!nvidia-smi

Mon Jul  4 08:45:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:1A:00.0 Off |                  Off |
| 33%   31C    P8    25W / 260W |   2100MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:3D:00.0 Off |                  Off |
| 33%   

In [37]:
features, labels = get_features_vgg(data_folder, patient_files)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 751/751 [14:56<00:00,  1.19s/it]


In [57]:
data_folder

'/Data2/hmd/data_split/murmur/train'

In [74]:
features['mel1'].shape

(2532, 128)

In [25]:
features['mel1'][0].shape

(64, 313, 1)

In [9]:
patient_files

['/Data2/hmd/data_split/murmur/train/2530.txt',
 '/Data2/hmd/data_split/murmur/train/9979.txt',
 '/Data2/hmd/data_split/murmur/train/9983.txt',
 '/Data2/hmd/data_split/murmur/train/13918.txt',
 '/Data2/hmd/data_split/murmur/train/14241.txt',
 '/Data2/hmd/data_split/murmur/train/14998.txt',
 '/Data2/hmd/data_split/murmur/train/23625.txt',
 '/Data2/hmd/data_split/murmur/train/24160.txt',
 '/Data2/hmd/data_split/murmur/train/29045.txt',
 '/Data2/hmd/data_split/murmur/train/29378.txt',
 '/Data2/hmd/data_split/murmur/train/31737.txt',
 '/Data2/hmd/data_split/murmur/train/36327.txt',
 '/Data2/hmd/data_split/murmur/train/39043.txt',
 '/Data2/hmd/data_split/murmur/train/39403.txt',
 '/Data2/hmd/data_split/murmur/train/39456.txt',
 '/Data2/hmd/data_split/murmur/train/40058.txt',
 '/Data2/hmd/data_split/murmur/train/40840.txt',
 '/Data2/hmd/data_split/murmur/train/43852.txt',
 '/Data2/hmd/data_split/murmur/train/44514.txt',
 '/Data2/hmd/data_split/murmur/train/45843.txt',
 '/Data2/hmd/data_split

In [16]:
 for i in range(num_patient_files):
    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    num_locations = get_num_locations(current_patient_data)
    recording_information = current_patient_data.split('\n')[1:num_locations+1]
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]#음성기록 
        filename = os.path.join(data_folder, recording_file)

In [17]:
current_patient_data

'85349 3 4000\nAV 85349_AV.hea 85349_AV.wav 85349_AV.tsv\nPV 85349_PV.hea 85349_PV.wav 85349_PV.tsv\nTV 85349_TV.hea 85349_TV.wav 85349_TV.tsv\n#Age: nan\n#Sex: Female\n#Height: nan\n#Weight: nan\n#Pregnancy status: True\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Normal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [18]:
num_locations

3

In [19]:
recording_information

['AV 85349_AV.hea 85349_AV.wav 85349_AV.tsv',
 'PV 85349_PV.hea 85349_PV.wav 85349_PV.tsv',
 'TV 85349_TV.hea 85349_TV.wav 85349_TV.tsv']

In [21]:
vgg_feature(filename).shape #길이가 다름.

(20, 128)

In [15]:
filename

NameError: name 'filename' is not defined

vgg 임베딩 코드

In [9]:
vggish_model = torch.hub.load('harritaylor/torchvggish', 'vggish')
def vgg_feature(fnm):
    vggish_model.eval()
    vggish_model.postprocess = False
    vggish_model.embeddings[5] = torch.nn.Sequential() # Remove last ReLU
    vggish_embeddings = vggish_model.forward(fnm).detach().cpu().numpy()
    return vggish_embeddings

Using cache found in /root/.cache/torch/hub/harritaylor_torchvggish_master


In [ ]:
num_patient_files = len(patient_files_trn)

In [ ]:
for i in range(num_patient_files):

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files_trn[i])
    num_locations = get_num_locations(current_patient_data)
    recording_information = current_patient_data.split('\n')[1:num_locations+1]
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)

        # Extract id
        id1 = recording_file.split('_')[0]
        features['id'].append(id1)

        # Extract melspec
        mel1 = feature_extract_melspec(filename)[0]
        features['mel1'].append(mel1)


In [22]:
mel1 = feature_extract_melspec(filename)[0]

In [23]:
mel1.shape

(64, 313)

In [1]:
filename

NameError: name 'filename' is not defined

In [ ]:
def get_features(data_folder, patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    labels = []

    classes = ['Present', 'Unknown', 'Absent']
    num_classes = len(classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = feature_extract_melspec(filename)[0]
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels and use one-hot encoding.
            current_labels = np.zeros(num_classes, dtype=int)
            label = get_label(current_patient_data)
            if label in classes:
                j = classes.index(label)
                current_labels[j] = 1
            labels.append(current_labels)
        
    M, N = features['mel1'][i].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    labels = np.array(labels)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    return features, labels

In [ ]:
current_murmur = np.zeros(num_murmur_classes, dtype=int)
    murmur = get_murmur(current_patient_data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    murmurs.append(current_murmur)

    current_outcome = np.zeros(num_outcome_classes, dtype=int)
    outcome = get_outcome(current_patient_data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1
    outcomes.append(current_outcome)


In [75]:
mel1 = vgg_feature(filename)

In [77]:
mel1.shape

(20, 128)

In [33]:
from tqdm import tqdm

In [10]:
def get_features_vgg(data_folder, patient_files_trn) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    murmurs=[] 
    outcomes=[]

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            mel1 = vgg_feature(filename)
            features['mel1'].append(mel1)

            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels and use one-hot encoding.
            current_murmur = np.zeros(num_murmur_classes, dtype=int)
            murmur = get_murmur(current_patient_data)
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[j] = 1
            murmurs.append(current_murmur)
            
            
            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome = get_outcome(current_patient_data)
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)
            
            
            
            
            
    #M, N = features['mel1'][i].shape
    #for i in range(len(features['mel1'])) :
    #    features['mel1'][i] = features['mel1'][i].reshape(M,N,1)

    murmurs = np.array(murmurs)
    outcomes = np.array(outcomes)
    
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    return features, murmurs, outcomes

In [99]:
features['mel1'][0].shape

(24, 128)

In [11]:
features,murmurs, outcomes= get_features_vgg(data_folder, patient_files)

100%|█████████████████████████████████████████| 751/751 [14:58<00:00,  1.20s/it]
/tmp/ipykernel_23066/3885234799.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features[k1] = np.array(features[k1])


In [90]:
features['mel1']

(15, 128)

In [12]:
#from tensorflow import keras
#from tensorflow.keras import layers

# -*- coding: utf-8 -*-
import argparse
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from tools import utils
from torch import Tensor
import torch.nn.functional as F



def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x

class Dense(nn.Module):
    
    def __init__(self,length):
        super(Dense, self).__init__()
        h1 = nn.Linear(length, 64)
        h2 = nn.Linear(64, 32)
        h3 = nn.Linear(32, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        if torch.cuda.is_available():
            self.hidden = self.hidden.cuda()
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1,1)
    
    
    
class cnn_ti(nn.Module):
    def __init__(self, classes_num):
        
        super(cnn_ti, self).__init__() 

        self.bn0 = nn.BatchNorm2d(64)
        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.dense_age = Dense(6)
        self.dense_sex = Dense(2)
        self.dense_hw = Dense(2)
        self.dense_preg = Dense(1)
        self.dense_loc = Dense(5)
        
        #self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024) 
        #self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(512, 507, bias=True)
        self.fc_audioset = nn.Linear(512, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
        
        
    def forward(self, input, age, sex, hw, preg, loc):

        """
        Input: (batch_size, data_length)"""

        #x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        #x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        #print(input.shape)
        #x = input.permute(1, 2, 0) 
        input = input.float()
        #x =input.unsqueeze(1)
        x = input.permute(0, 2,1, 3) 
        #print(x.shape)
        #x = x.transpose(1, 3)
        #print(x.shape)
        x = self.bn0(x)
        #print(x.shape)
        x = x.permute(0, 2,1, 3) 
        #x = x.transpose(1, 3)
        print(x.shape)
        
        x = self.conv_block1(x, pool_size=(1, 2), pool_type='avg')
        print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        print(x.shape)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        print(x.shape)
        #print(x)
        x = torch.mean(x, dim=3)
        #print(x)
        #print(x.shape)
        (x1, _) = torch.max(x, dim=2)
        print(x1.shape)
        x2 = torch.mean(x, dim=2)
        print(x2.shape)
        x = x1 + x2
        #print(x.shape)
        x = F.dropout(x, p=0.5, training=self.training)
        print(x.shape)
        x = F.relu_(self.fc1(x))
        print('fc통과:',x.shape)
        age_ = self.dense_age(age)
        sex_ = self.dense_sex(sex)
        hw_ = self.dense_hw(hw)
        preg_ = self.dense_preg(preg)
        loc_ = self.dense_loc(loc)
        
        #print(x.shape)
        cat = torch.cat([x, age_,sex_,hw_,preg_,loc_],axis = 1)
        print(cat.shape)
        #embedding = F.dropout(x, p=0.5, training=self.training)
        out = self.fc_audioset(cat)
        #print(out.shape)
        #clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return out


In [87]:
label = np.empty(len(murmurs))
for i,l in enumerate(murmurs) :
    label[i] = int(list(l).index(1))

In [88]:
label

array([2., 2., 2., ..., 2., 2., 2.])

In [15]:
vggish_model = torch.hub.load('harritaylor/torchvggish', 'vggish')

Using cache found in /root/.cache/torch/hub/harritaylor_torchvggish_master


In [16]:
vggish_model.eval()
vggish_model.postprocess = False
vggish_model.embeddings[5] = torch.nn.Sequential() # Remove last ReLU

In [17]:
vggish_embeddings = vggish_model.forward(filename).detach().cpu().numpy()

In [18]:
vggish_embeddings.shape # t.melbin

(20, 128)

In [ ]:
Sig=[]

for i in range


In [43]:
model = cnn_ti(3)

In [46]:
x = torch.from_numpy(vggish_embeddings) #torch로 변환

In [47]:
x.shape

torch.Size([20, 128])

In [49]:
x = x.unsqueeze(0)

In [50]:
x.shape

torch.Size([1, 20, 128])

In [52]:
model

cnn_ti(
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [ ]:
audio_feature, age, sex, hw, preg, loc, data_label_torch =next(iter(dataloader_train_murmurs)) 

In [44]:
vggish_embeddings=vggish_embeddings.unsqueeze(0)

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [ ]:
model(vggish_embeddings)

In [13]:
import numpy as np
import torch
import sys
import os
sys.path.append("..")
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn


from scipy.io import wavfile
import random


class myDataset(Dataset):
    def __init__(self, x_npy, y_npy):    # spec or mel_spec
        self.x_npy = x_npy
        self.y_npy = y_npy

        
    def __getitem__(self, idx):
        cur_idx = idx
        while 1:
            try:
                data_feature = self.x_npy['mel1'][cur_idx]
                age = self.x_npy['age'][cur_idx]
                sex = self.x_npy['sex'][cur_idx]
                hw = self.x_npy['hw'][cur_idx]
                if self.x_npy['preg'][cur_idx] :
                    preg = np.array([1])
                else:
                    preg = np.array([0])
                
                loc = self.x_npy['loc'][cur_idx]
                break
            except:
                cur_idx += 1 
        
        data_label = self.y_npy[cur_idx]
        return data_feature,age,sex,hw,preg,loc, data_label

    def __len__(self):
        return self.x_npy['mel1'].shape[0]

def myCollateFn(sample_batch):
    sample_batch = sorted(sample_batch, key=lambda x: x[0].shape[0], reverse=True)
    #data_feature = [spec_augment(torch.from_numpy(x[0])) for x in sample_batch]#spec
    data_feature =[torch.from_numpy(x[0]) for x in sample_batch]
    age = [torch.from_numpy(x[1]) for x in sample_batch]
    sex = [torch.from_numpy(x[2]) for x in sample_batch]
    hw = [torch.from_numpy(x[3]) for x in sample_batch]
    preg = [torch.from_numpy(x[4]) for x in sample_batch]
    loc = [torch.from_numpy(x[5]) for x in sample_batch]
    
    data_feature = pad_sequence(data_feature, batch_first=False, padding_value=0.0)
    age = pad_sequence(age, batch_first=False, padding_value=0.0)
    sex = pad_sequence(sex, batch_first=False, padding_value=0.0)
    hw = pad_sequence(hw, batch_first=False, padding_value=0.0)
    preg = pad_sequence(preg, batch_first=False, padding_value=0.0)
    loc = pad_sequence(loc, batch_first=False, padding_value=0.0)
    
    data_label = torch.tensor([x[-1] for x in sample_batch]).unsqueeze(-1)
    return data_feature, age, sex, hw, preg, loc, data_label

class myDataLoader(DataLoader):
    def __init__(self, *args, **kwargs):
        super(myDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = myCollateFn


In [240]:
#from tensorflow import keras
#from tensorflow.keras import layers

# -*- coding: utf-8 -*-
import argparse
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from tools import utils
from torch import Tensor
import torch.nn.functional as F



def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x

class Dense(nn.Module):
    
    def __init__(self,length):
        super(Dense, self).__init__()
        h1 = nn.Linear(length, 64)
        h2 = nn.Linear(64, 32)
        h3 = nn.Linear(32, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        if torch.cuda.is_available():
            self.hidden = self.hidden.cuda()
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1,1)
    
    
    
class vgg_hj(nn.Module):
    def __init__(self, classes_num):
        
        super(vgg_hj, self).__init__() 
        


        self.bn0 = nn.BatchNorm2d(64)
        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.dense_age = Dense(6)
        self.dense_sex = Dense(2)
        self.dense_hw = Dense(2)
        self.dense_preg = Dense(1)
        self.dense_loc = Dense(5)
        
        #self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024) 
        #self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(512, 507, bias=True)
        self.fc_audioset = nn.Linear(512, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
        
        
    def forward(self, input, age, sex, hw, preg, loc):

        """
        Input: (batch_size, data_length)"""

        #x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        #x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        #print(input.shape)
        #x = input.permute(1, 2, 0) 
        input = input.float()
        #x =input.unsqueeze(1)
        x = input.permute(0, 2,1, 3) 
        #print(x.shape)
        #x = x.transpose(1, 3)
        #print(x.shape)
        x = self.bn0(x)
        #print(x.shape)
        x = x.permute(0, 2,1, 3) 
        #x = x.transpose(1, 3)
        #print(x.shape)
        
        x = self.conv_block1(x, pool_size=(1, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        #print(x)
        x = torch.mean(x, dim=3)
        #print(x)
        #print(x.shape)
        (x1, _) = torch.max(x, dim=2)
        #print(x1.shape)
        x2 = torch.mean(x, dim=2)
        #print(x2.shape)
        x = x1 + x2
        #print(x.shape)
        x = F.dropout(x, p=0.5, training=self.training)
        #print(x.shape)
        x = F.relu_(self.fc1(x))
        #print('fc통과:',x.shape)
        age_ = self.dense_age(age)
        sex_ = self.dense_sex(sex)
        hw_ = self.dense_hw(hw)
        preg_ = self.dense_preg(preg)
        loc_ = self.dense_loc(loc)
        
        #print(x.shape)
        cat = torch.cat([x, age_,sex_,hw_,preg_,loc_],axis = 1)
        #print(cat.shape)
        #embedding = F.dropout(x, p=0.5, training=self.training)
        out = self.fc_audioset(cat)
        #print(out.shape)
        #clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return out


In [53]:
outcome_classes = ['Abnormal', 'Normal']

In [ ]:
data_folder, patient_files

In [58]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [59]:
current_recordings

[array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
 array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
 array([ 2276,  2343,  2448, ..., -2732, -5634, -4290], dtype=int16),
 array([12672, 10041,  3215, ...,  -658,     8,   528], dtype=int16)]

In [62]:
current_recordings

[array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
 array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
 array([ 2276,  2343,  2448, ..., -2732, -5634, -4290], dtype=int16),
 array([12672, 10041,  3215, ...,  -658,     8,   528], dtype=int16)]

In [63]:
num_patient_files

751

In [247]:
#from tensorflow import keras
#from tensorflow.keras import layers

# -*- coding: utf-8 -*-
import argparse
import os
import time
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sys
import random
from tqdm import tqdm
from tools import utils
from torch import Tensor
import torch.nn.functional as F



def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)
 
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
    
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(ConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()
        
    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

        
    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')
        
        return x

class Dense(nn.Module):
    
    def __init__(self,length):
        super(Dense, self).__init__()
        h1 = nn.Linear(length, 64)
        h2 = nn.Linear(64, 32)
        h3 = nn.Linear(32, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        if torch.cuda.is_available():
            self.hidden = self.hidden.cuda()
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1,1)

In [248]:
class vgg_hj(nn.Module):
    def __init__(self, classes_num):
        
        super(vgg_hj, self).__init__() 
        


        self.bn0 = nn.BatchNorm2d(128)
        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.dense_age = Dense(6)
        self.dense_sex = Dense(2)
        self.dense_hw = Dense(2)
        self.dense_preg = Dense(1)
        self.dense_loc = Dense(5)
        
        #self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024) 
        #self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(512, 507, bias=True)
        self.fc_audioset = nn.Linear(512, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
        
        
    def forward(self, input, age, sex, hw, preg, loc):

        """
        Input: (batch_size, data_length)"""

        #x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        #x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        #print(input.shape)
        #x = input.permute(1, 2, 0) 
        input = input.float()
        x =input.unsqueeze(1)
        x = x.permute(2, 3,0, 1) 
        print(x.shape)
        #x = x.transpose(1, 3)
        #print(x.shape)
        x = self.bn0(x)
        #print(x.shape)
        x = x.permute(0, 2,1, 3) 
        x = x.transpose(1, 3)
        print(x.shape)
        
        x = self.conv_block1(x, pool_size=(1, 2), pool_type='avg')
        print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        print(x.shape)
        x = self.conv_block2(x, pool_size=(1, 2), pool_type='avg')
        print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #print(x.shape)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        #print(x.shape)
        x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        #x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        #x = F.dropout(x, p=0.2, training=self.training)
        print(x.shape)
        #print(x)
        x = torch.mean(x, dim=3)
        #print(x)
        #print(x.shape)
        (x1, _) = torch.max(x, dim=2)
        print(x1.shape)
        x2 = torch.mean(x, dim=2)
        print(x2.shape)
        x = x1 + x2
        #print(x.shape)
        x = F.dropout(x, p=0.5, training=self.training)
        print(x.shape)
        x = F.relu_(self.fc1(x))
        print('fc통과:',x.shape)#1,507
       
        age_ = self.dense_age(age)
        sex_ = self.dense_sex(sex)
        hw_ = self.dense_hw(hw)
        preg_ = self.dense_preg(preg)
        loc_ = self.dense_loc(loc)
        
        #print(x.shape)
        cat = torch.cat([x, age_,sex_,hw_,preg_,loc_],axis = 1)
        print(cat.shape)
        #embedding = F.dropout(x, p=0.5, training=self.training)
        out = self.fc_audioset(cat)
        #print(out.shape)
        #clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        #output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return out


7

In [71]:
murmurs

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [72]:
dataset_train = myDataset(features, murmurs)
dataloader_train = myDataLoader(dataset=dataset_train,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

In [73]:
x = next(iter(dataloader_train))

In [85]:
x[-1].unsqueeze(-1)

tensor([[[[1]],

         [[0]],

         [[0]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[0]],

         [[1]],

         [[0]]],


        [[[1]],

         [[0]],

         [[0]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[0]],

         [[1]],

         [[0]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[0]],

         [[1]],

         [[0]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[1]],

         [[0]],

         [[0]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[0]],

         [[0]],

         [[1]]],


        [[[1]],

         [[0]],

         [[0]]],


        [[[1]],

         [[0]],

         [[0]]],


        [[[1]],

         [[0]],

         [[0]]],


        [[[1]],

         [[0]],

         [[0]]],


        [[[0]],

         [[0]],

         [[1

In [107]:
nnet = vgg_hj(num_murmur_classes)

In [106]:
num_murmur_classes = len(murmur_classes)

In [105]:
murmur_classes = ['Present', 'Unknown', 'Absent']

In [257]:
# Train your model.
def train_challenge_model(data_folder, model_folder, verbose):
    gc.collect()
    torch.cuda.empty_cache()
    GPU_NUM = 0 # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
#    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device)
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    
    #features = np.vstack(features)
    #murmurs = np.vstack(murmurs)
    #outcomes = np.vstack(outcomes)
    # Train the model.
    log_dir = './log/'
    project = 'hj0'
    logger = get_logger(log_dir + '/' + project)
    
    #label 
    murmurs_label = np.empty(len(murmurs))
    for i,l in enumerate(murmurs) :
        murmurs_label[i] = int(list(l).index(1))
    
    outcomes_label = np.empty(len(outcomes))
    for i,l in enumerate(outcomes) :
        outcomes_label[i] = int(list(l).index(1))
    
    
    dataset_train_murmurs = myDataset(features,murmurs_label)
    dataloader_train_murmurs = myDataLoader(dataset=dataset_train_murmurs,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)
    
    
    dataset_train_outcomes = myDataset(features,outcomes_label)
    dataloader_train_outcoms = myDataLoader(dataset=dataset_train_outcomes,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

    all_file_train = len(dataloader_train_murmurs)
    # Define model
    nnet = vgg_hj(num_murmur_classes) 
    nnet = nnet.cuda()
    optimizer = optim.Adam(nnet.parameters(), lr=4e-4)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.91)
    criterion = nn.CrossEntropyLoss()
    # Train the model.
    if verbose >= 1:
        print('Training model...')
    imputer = SimpleImputer().fit(features['hw'])
    features['hw'] = imputer.transform(features['hw'])
    
    
    for iter_ in range(15):  # args.end_iter
        start_time = time.time()
        running_loss = 0.0
        train_epoch_acc = 0.0
        nnet.train()
        labels=[]
        idx = 0
#    epoch[str(iter_)] = {}
        for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_murmurs):  
            #print(audio_feature.shape)
        
            idx = idx + 1
            #audio_feature = torch.nan_to_num(audio_feature, nan=0)
            audio_feature[audio_feature != audio_feature] = 0
        
            #audio_feature = audio_feature.permute(1,3,0,2)
            #print(audio_feature.shape)
            audio_feature = audio_feature.cuda()
            age = age.permute(1,0).float().cuda()
            #print(age.shape)
            sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
            hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
            preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
            loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
            data_label_torch = data_label_torch.cuda()
                
            data_label_torch = data_label_torch.long()
            outputs = nnet(input=audio_feature, age = age , sex = sex, hw = hw, preg = preg, loc = loc)
            
            #softmax = nn.Softmax(dim = 1)
            
            #outputs = softmax(outputs)
            
            #labels = np.argmax(outputs.detach().cpu().numpy())
            
            
            loss = criterion(outputs.float(), data_label_torch.squeeze(1))
            #train_acc = compute_weighted_accuracy(data_label_torch.squeeze(1), outputs,murmur_classes )
            train_acc = multi_acc(outputs, data_label_torch.squeeze(1))
            #print(outputs.shape)
            #print(data_label_torch.squeeze(1).shape)
            #print(train_acc)
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
        logger.info("Iteration:{0}, loss = {1:.6f}, acc = {2:.3f} ".format(iter_, running_loss/all_file_train, train_epoch_acc/all_file_train))
    
    save_challenge_model(model_folder, murmur_classes, nnet,m_name='vgg_hj')
                

In [256]:
def save_challenge_model(model_folder,murmur_classes, model,m_name):
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename = os.path.join(model_folder, m_name + '_model.hdf5')
    torch.save(model.state_dict(), filename)
    d = {'murmur_classes': murmur_classes, 'murmur_classifier': model}
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)

In [ ]:
save_challenge_model(model_folder, nnet, classes, m_name = 'toy_ti', mel_shape = (100, 313, 1) )

In [154]:
save_challenge_model

<function __main__.save_challenge_model(model_folder, m_name, murmur_classes, murmur_classifier)>

In [29]:
#outcome 
nn.BCEWithLogitsLoss()

BCEWithLogitsLoss()

In [219]:
data_label_torch[0][0]

tensor(0, device='cuda:0')

In [217]:
labels = np.zeros((64,3), dtype=np.int_)

In [232]:
murmurs

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [226]:
len(data_label_torch)

64

In [227]:
for idx in range(len(data_label_torch)):
    labels[0][idx]= data_label_torch[idx][0]

IndexError: index 3 is out of bounds for axis 0 with size 3

In [230]:
labels

array([[0, 2, 2],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0,

In [229]:
data_label_torch[0][0]

tensor(0, device='cuda:0')

In [189]:
data_label_torch

tensor([[0.],
        [2.],
        [2.],
        [1.],
        [0.],
        [2.],
        [2.],
        [1.],
        [2.],
        [1.],
        [2.],
        [0.],
        [2.],
        [2.],
        [0.],
        [0.],
        [0.],
        [0.],
        [2.],
        [2.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [2.],
        [2.],
        [2.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [2.],
        [2.],
        [0.],
        [1.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [1.],
        [2.],
        [2.],
        [2.],
        [0.],
        [0.],
        [2.],
        [0.]], dtype=torch.float64)

In [ ]:
save_challenge_model

In [45]:
murmurs

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [46]:
murmurs_label.

array([2., 2., 2., ..., 2., 2., 2.])

In [75]:
murmurs_label = np.empty(len(murmurs))
for i,l in enumerate(murmurs) :
    murmurs_label[i] = int(list(l).index(1))
    
outcomes_label = np.empty(len(outcomes))
for i,l in enumerate(outcomes) :
    outcomes_label[i] = int(list(l).index(1))

In [188]:
murmurs_label

array([2., 2., 2., ..., 2., 2., 2.])

In [76]:
dataset_train_murmurs = myDataset(features,murmurs_label)
dataloader_train_murmurs = myDataLoader(dataset=dataset_train_murmurs,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

In [34]:
murmur_classes = ['Present', 'Unknown', 'Absent']
num_murmur_classes = len(murmur_classes)

In [35]:
nnet = vgg_hj(num_murmur_classes) 
nnet = nnet.cuda()
optimizer = optim.Adam(nnet.parameters(), lr=4e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.91)

criterion = nn.CrossEntropyLoss()

In [187]:
audio_feature, age, sex, hw, preg, loc, data_label_torch = next(iter(dataloader_train_murmurs))

In [206]:
data_label_torch = data_label_torch.cuda()
                
data_label_torch = data_label_torch.long()

In [207]:
data_label_torch 

tensor([[0],
        [2],
        [2],
        [1],
        [0],
        [2],
        [2],
        [1],
        [2],
        [1],
        [2],
        [0],
        [2],
        [2],
        [0],
        [0],
        [0],
        [0],
        [2],
        [2],
        [0],
        [1],
        [0],
        [0],
        [0],
        [2],
        [2],
        [2],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [2],
        [2],
        [0],
        [1],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [1],
        [2],
        [2],
        [2],
        [0],
        [0],
        [2],
        [0]], device='cuda:0')

In [78]:
audio_feature = audio_feature.cuda()
age = age.permute(1,0).float().cuda()
            #print(age.shape)
sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
data_label_torch = data_label_torch.cuda()
                
data_label_torch = data_label_torch.long()
outputs = nnet(input=audio_feature, age = age , sex = sex, hw = hw, preg = preg, loc = loc)

torch.Size([64, 128, 34, 1])
torch.Size([64, 1, 128, 34])
torch.Size([64, 64, 128, 17])
torch.Size([64, 64, 128, 17])
torch.Size([64, 128, 128, 8])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


In [58]:
murmurs.shape

(2532, 3)

In [85]:
np.argmax(outputs.detach().cpu().numpy()[0]) # label

0

In [87]:
softmax = nn.Softmax(dim = 1)
outputs = softmax(outputs)
    
prob1 = outputs.mean(axis = 0)
    
prob1 = prob1.data.cpu().numpy()

In [104]:
outputs[0]

tensor([0.3853, 0.2572, 0.3575], device='cuda:0', grad_fn=<SelectBackward>)

In [ ]:
outputs[0]

In [ ]:
labels = np.zeros(3, dtype=np.int_)
labels[idx] = 1

In [88]:
prob1

array([0.40404636, 0.25722697, 0.3387267 ], dtype=float32)

In [95]:
outputs

tensor([[0.3853, 0.2572, 0.3575],
        [0.4270, 0.2319, 0.3411],
        [0.3851, 0.2847, 0.3302],
        [0.3279, 0.2757, 0.3964],
        [0.5627, 0.2178, 0.2194],
        [0.3402, 0.3603, 0.2995],
        [0.5247, 0.2314, 0.2438],
        [0.4454, 0.2543, 0.3003],
        [0.4354, 0.2383, 0.3263],
        [0.2537, 0.3609, 0.3854],
        [0.5419, 0.2309, 0.2272],
        [0.2717, 0.4402, 0.2881],
        [0.4025, 0.3599, 0.2376],
        [0.3929, 0.2348, 0.3723],
        [0.2607, 0.2236, 0.5157],
        [0.5612, 0.2177, 0.2211],
        [0.5098, 0.2283, 0.2620],
        [0.5145, 0.2363, 0.2492],
        [0.3438, 0.2432, 0.4130],
        [0.3287, 0.2527, 0.4186],
        [0.2597, 0.2335, 0.5068],
        [0.4649, 0.2315, 0.3035],
        [0.5163, 0.2237, 0.2599],
        [0.3802, 0.3440, 0.2758],
        [0.5036, 0.2404, 0.2560],
        [0.5578, 0.2159, 0.2263],
        [0.5755, 0.2121, 0.2123],
        [0.4593, 0.2539, 0.2867],
        [0.5738, 0.2125, 0.2137],
        [0.540

In [123]:
data_label_torch.squeeze(1).shape

torch.Size([64])

In [99]:
outputs

tensor([[0.3853, 0.2572, 0.3575],
        [0.4270, 0.2319, 0.3411],
        [0.3851, 0.2847, 0.3302],
        [0.3279, 0.2757, 0.3964],
        [0.5627, 0.2178, 0.2194],
        [0.3402, 0.3603, 0.2995],
        [0.5247, 0.2314, 0.2438],
        [0.4454, 0.2543, 0.3003],
        [0.4354, 0.2383, 0.3263],
        [0.2537, 0.3609, 0.3854],
        [0.5419, 0.2309, 0.2272],
        [0.2717, 0.4402, 0.2881],
        [0.4025, 0.3599, 0.2376],
        [0.3929, 0.2348, 0.3723],
        [0.2607, 0.2236, 0.5157],
        [0.5612, 0.2177, 0.2211],
        [0.5098, 0.2283, 0.2620],
        [0.5145, 0.2363, 0.2492],
        [0.3438, 0.2432, 0.4130],
        [0.3287, 0.2527, 0.4186],
        [0.2597, 0.2335, 0.5068],
        [0.4649, 0.2315, 0.3035],
        [0.5163, 0.2237, 0.2599],
        [0.3802, 0.3440, 0.2758],
        [0.5036, 0.2404, 0.2560],
        [0.5578, 0.2159, 0.2263],
        [0.5755, 0.2121, 0.2123],
        [0.4593, 0.2539, 0.2867],
        [0.5738, 0.2125, 0.2137],
        [0.540

In [86]:
softmax = nn.Softmax(dim = 1)
outputs = softmax(outputs)
    
prob1 = outputs.mean(axis = 0)
    
prob1 = prob1.data.cpu().numpy()
    
    
    
    
    
#    print(prob1.shape)
    
#    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

    # Get classifier probabilities.
#    prob1 = res1.mean(axis = 0) ## simple rule for now
idx = np.argmax(outputs)
    # Choose label with higher probability.
labels = np.zeros(len(classes), dtype=np.int_)
labels[idx] = 1

NameError: name 'classes' is not defined

In [163]:
idx = np.argmax(outputs[0].detach().cpu().numpy())

In [162]:
del labels

In [164]:
labels = np.zeros(3, dtype=np.int_)
labels[idx] = 1

In [165]:
labels

array([1, 0, 0])

In [178]:
data_label_torch[1]

tensor([2], device='cuda:0')

In [176]:
label_ = np.zeros(3, dtype=np.int_)
label_[data_label_torch[1]]=1

In [177]:
label_

array([0, 0, 1])

In [170]:
outputs[:,0]

tensor([0.3853, 0.4270, 0.3851, 0.3279, 0.5627, 0.3402, 0.5247, 0.4454, 0.4354,
        0.2537, 0.5419, 0.2717, 0.4025, 0.3929, 0.2607, 0.5612, 0.5098, 0.5145,
        0.3438, 0.3287, 0.2597, 0.4649, 0.5163, 0.3802, 0.5036, 0.5578, 0.5755,
        0.4593, 0.5738, 0.5400, 0.5421, 0.5526, 0.4708, 0.2952, 0.3614, 0.4195,
        0.2450, 0.3684, 0.4947, 0.4577, 0.4566, 0.2517, 0.3443, 0.2187, 0.2447,
        0.2480, 0.2176, 0.3020, 0.4964, 0.2890, 0.4424, 0.2792, 0.2957, 0.2776,
        0.4393, 0.5632, 0.4964, 0.5199, 0.3383, 0.5043, 0.4393, 0.3698, 0.2443,
        0.3264], device='cuda:0', grad_fn=<SelectBackward>)

In [103]:
outputs.detach().cpu().numpy()

array([[0.38526216, 0.257211  , 0.35752684],
       [0.42696187, 0.23190613, 0.34113193],
       [0.38505796, 0.2847491 , 0.3301929 ],
       [0.32790118, 0.27574858, 0.3963502 ],
       [0.5627348 , 0.21784575, 0.21941945],
       [0.34019533, 0.36029583, 0.29950884],
       [0.5247269 , 0.2314361 , 0.243837  ],
       [0.44539705, 0.25432134, 0.30028158],
       [0.43542477, 0.23829024, 0.32628495],
       [0.25373155, 0.3609107 , 0.38535774],
       [0.54185414, 0.23090911, 0.2272367 ],
       [0.27172163, 0.44016278, 0.28811562],
       [0.40252703, 0.35985273, 0.23762017],
       [0.39293835, 0.2347685 , 0.37229314],
       [0.26067933, 0.22357851, 0.5157422 ],
       [0.5612148 , 0.21773253, 0.22105263],
       [0.50975186, 0.22829086, 0.26195726],
       [0.5145244 , 0.23630229, 0.2491733 ],
       [0.34382272, 0.24318837, 0.41298893],
       [0.3287089 , 0.2526513 , 0.4186398 ],
       [0.25971118, 0.23347774, 0.5068111 ],
       [0.46492252, 0.23154938, 0.3035281 ],
       [0.

In [84]:
outputs

tensor([[ 0.2285, -1.4845,  0.0641],
        [ 1.8936, -4.3125,  1.4363],
        [-0.3833, -1.3562, -0.7643],
        [ 0.5407, -0.2107,  0.9970],
        [ 1.9575, -2.2326, -1.8317],
        [-0.0910,  0.0583, -0.5328],
        [ 1.5309, -1.4650, -0.6712],
        [ 1.6120, -0.3546,  0.6829],
        [ 2.0264, -1.1012,  1.4145],
        [-0.4176,  1.3053,  1.4475],
        [ 1.3385, -1.4765, -1.8260],
        [-0.9681,  0.4556, -0.6440],
        [ 0.2031, -0.0264, -3.1403],
        [ 2.3220, -1.8637,  2.2131],
        [ 0.6018, -3.1901,  2.2778],
        [ 2.2294, -2.0972, -1.3116],
        [ 3.3409, -0.3908,  1.6859],
        [ 1.8482, -0.8482, -0.1843],
        [ 0.5708, -1.6667,  0.9579],
        [-1.1437, -2.6266, -0.6070],
        [ 0.5302, -0.7772,  2.2494],
        [ 1.0810, -3.1065,  0.1587],
        [ 2.5195, -2.6777,  0.8248],
        [ 1.5948,  1.3579,  0.4632],
        [ 1.1415, -1.3615, -0.6931],
        [ 3.6383, -2.9549,  0.6577],
        [ 5.7888, -2.4904, -0.8487],
 

In [65]:
data_label_torch.squeeze(2)

tensor([[1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 

In [ ]:
loss = criterion(outputs.float(), data_label_torch.squeeze(1))

In [44]:
data_label_torch.squeeze(1).shape

torch.Size([64])

In [51]:
dataset_train_murmurs = myDataset(features,murmurs)
dataloader_train_murmurs = myDataLoader(dataset=dataset_train_murmurs,
                                    batch_size=64,
                                    shuffle=False,
                                    num_workers=0)

In [55]:
dataset_train_murmurs[2]

(array([[-0.26812872,  0.1456995 ,  0.2229015 , ..., -0.37224373,
          0.06185362, -0.20789492],
        [-0.3532718 , -0.06782424,  0.15311153, ..., -0.53001344,
          0.15046059, -0.30170864],
        [-0.15403742,  0.12375794,  0.19781077, ..., -0.2646627 ,
          0.1298432 , -0.09531417],
        ...,
        [-0.1602506 ,  0.27430457,  0.12100403, ..., -0.3193402 ,
          0.07736264, -0.19237593],
        [-0.2800865 , -0.12960649, -0.07742852, ..., -0.61623394,
          0.07834175, -0.4003654 ],
        [-0.33421162, -0.00154565,  0.26805216, ..., -0.5752419 ,
         -0.07102329, -0.20493415]], dtype=float32),
 array([0, 0, 1, 0, 0, 0]),
 array([1, 0]),
 array([98. , 15.9]),
 array([0]),
 array([0., 0., 0., 1., 0.]),
 array([0, 0, 1]))

In [69]:
data_label_torch.squeeze(2)

tensor([[1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 

In [70]:
outputs

tensor([[ 1.7412e+00, -2.3447e+00, -4.8400e-01],
        [-5.1983e-01, -3.1038e+00,  1.8714e+00],
        [ 1.6744e+00, -3.9311e+00, -1.6019e+00],
        [-9.0416e-01, -6.9645e-01, -1.2474e+00],
        [ 1.7310e+00, -3.3657e+00, -3.7368e+00],
        [ 3.5154e+00, -2.7164e+00,  4.4905e-01],
        [ 3.6265e+00, -2.2319e+00, -4.1153e-01],
        [ 1.9146e+00, -4.9297e-01,  1.2156e+00],
        [ 4.7234e-01, -2.2077e-01,  1.5240e+00],
        [-1.3380e+00, -2.1316e+00, -2.9917e-01],
        [ 3.3243e+00, -1.3581e+00,  2.2266e+00],
        [ 2.0434e+00, -4.5338e+00,  5.9820e-01],
        [ 8.3841e-01,  1.0704e+00, -1.0717e-01],
        [ 6.2175e-01, -1.6871e+00, -7.0502e-01],
        [-1.0158e+00, -3.2028e+00, -1.3146e+00],
        [-1.8057e-02, -3.1254e+00,  1.3839e+00],
        [-7.6661e-01, -3.3410e+00,  9.8358e-02],
        [ 9.6963e-01, -5.3035e-01,  1.8502e+00],
        [ 1.7504e+00, -4.1564e+00,  8.2769e-01],
        [ 3.5583e+00, -1.9731e+00,  1.6008e+00],
        [-1.6832e+00

In [130]:
np.argmax(outputs)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [179]:
compute_weighted_accuracy(label_,labels,murmur_classes)

IndexError: tuple index out of range

In [180]:
label_

array([0, 0, 1])

In [141]:
np.argmax(outputs[0].detach().cpu().numpy())

0

In [156]:
labels

array([1, 0, 0])

In [147]:
np.shape(labels)[0]

3

In [150]:
outputs[0]

tensor([[0.3853, 0.2572, 0.3575],
        [0.4270, 0.2319, 0.3411],
        [0.3851, 0.2847, 0.3302],
        [0.3279, 0.2757, 0.3964],
        [0.5627, 0.2178, 0.2194],
        [0.3402, 0.3603, 0.2995],
        [0.5247, 0.2314, 0.2438],
        [0.4454, 0.2543, 0.3003],
        [0.4354, 0.2383, 0.3263],
        [0.2537, 0.3609, 0.3854],
        [0.5419, 0.2309, 0.2272],
        [0.2717, 0.4402, 0.2881],
        [0.4025, 0.3599, 0.2376],
        [0.3929, 0.2348, 0.3723],
        [0.2607, 0.2236, 0.5157],
        [0.5612, 0.2177, 0.2211],
        [0.5098, 0.2283, 0.2620],
        [0.5145, 0.2363, 0.2492],
        [0.3438, 0.2432, 0.4130],
        [0.3287, 0.2527, 0.4186],
        [0.2597, 0.2335, 0.5068],
        [0.4649, 0.2315, 0.3035],
        [0.5163, 0.2237, 0.2599],
        [0.3802, 0.3440, 0.2758],
        [0.5036, 0.2404, 0.2560],
        [0.5578, 0.2159, 0.2263],
        [0.5755, 0.2121, 0.2123],
        [0.4593, 0.2539, 0.2867],
        [0.5738, 0.2125, 0.2137],
        [0.540

In [67]:
outputs.shape

torch.Size([64, 3])

In [74]:
outputs.detach().cpu().numpy()

array([[ 1.74118125e+00, -2.34474921e+00, -4.83996868e-01],
       [-5.19827724e-01, -3.10376549e+00,  1.87137878e+00],
       [ 1.67438555e+00, -3.93106389e+00, -1.60185516e+00],
       [-9.04158950e-01, -6.96449280e-01, -1.24741912e+00],
       [ 1.73096657e+00, -3.36571002e+00, -3.73683977e+00],
       [ 3.51535273e+00, -2.71639967e+00,  4.49049413e-01],
       [ 3.62653971e+00, -2.23185706e+00, -4.11525726e-01],
       [ 1.91455400e+00, -4.92971212e-01,  1.21561074e+00],
       [ 4.72339749e-01, -2.20769346e-01,  1.52400374e+00],
       [-1.33798409e+00, -2.13157940e+00, -2.99171746e-01],
       [ 3.32426023e+00, -1.35805535e+00,  2.22655392e+00],
       [ 2.04335165e+00, -4.53378868e+00,  5.98198414e-01],
       [ 8.38408887e-01,  1.07036185e+00, -1.07173055e-01],
       [ 6.21749282e-01, -1.68705130e+00, -7.05020308e-01],
       [-1.01577783e+00, -3.20283747e+00, -1.31457114e+00],
       [-1.80573463e-02, -3.12538695e+00,  1.38385272e+00],
       [-7.66605973e-01, -3.34095573e+00

In [235]:
outputs

NameError: name 'output' is not defined

In [ ]:
compute_weighted_accuracy(murmurs)

In [19]:
def compute_weighted_accuracy(labels, outputs, classes):
    # Define constants.
    if classes == ['Present', 'Unknown', 'Absent']:
        weights = np.array([[5, 3, 1], [5, 3, 1], [5, 3, 1]])
    elif classes == ['Abnormal', 'Normal']:
        weights = np.array([[5, 1], [5, 1]])
    else:
        raise NotImplementedError('Weighted accuracy undefined for classes {}'.format(', '.join(classes)))

    # Compute confusion matrix.
    assert(np.shape(labels) == np.shape(outputs))
    A = compute_confusion_matrix(labels, outputs)

    # Multiply the confusion matrix by the weight matrix.
    assert(np.shape(A) == np.shape(weights))
    B = weights * A

    # Compute weighted_accuracy.
    if np.sum(B) > 0:
        weighted_accuracy = np.trace(B) / np.sum(B)
    else:
        weighted_accuracy = float('nan')

    return weighted_accuracy

In [28]:
def compute_confusion_matrix(labels, outputs):
    assert(np.shape(labels)[0] == np.shape(outputs)[0])
    assert(all(value in (0, 1, True, False) for value in np.unique(labels)))
    assert(all(value in (0, 1, True, False) for value in np.unique(outputs)))

    num_patients = np.shape(labels)[0]
    num_label_classes = np.shape(labels)[1]
    num_output_classes = np.shape(outputs)[1]

    A = np.zeros((num_output_classes, num_label_classes))
    for k in range(num_patients):
        for i in range(num_output_classes):
            for j in range(num_label_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

In [151]:
np.shape(labels)[0]

3

In [152]:
np.shape(labels)[1]

IndexError: tuple index out of range

In [258]:
import sys
from helper_code import is_integer
#from team_code import train_challenge_model

if __name__ == '__main__':
    # Parse the arguments.
    if not (len(sys.argv) == 3 or len(sys.argv) == 4):
        raise Exception('Include the data and model folders as arguments, e.g., python train_model.py data model.')

    # Define the data and model foldes.
    train_folder = '/Data2/hmd/data_split/murmur/train'
    #test_folder ='/Data2/hmd/data_split/murmur/test'
    model_folder = 'model'

    # Change the level of verbosity; helpful for debugging.
    if len(sys.argv)==4 and is_integer(sys.argv[3]):
        verbose = int(sys.argv[3])
    else:
        verbose = 1

        
        
        
    train_challenge_model(train_folder, model_folder, verbose) ### Teams: Implement this function!!!

Finding data files...
Extracting features and labels from the Challenge data...
Training model...


  0%|                                                    | 0/40 [00:00<?, ?it/s]

torch.Size([64, 128, 34, 1])
torch.Size([64, 1, 128, 34])
torch.Size([64, 64, 128, 17])
torch.Size([64, 64, 128, 17])
torch.Size([64, 128, 128, 8])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


  2%|█                                           | 1/40 [00:00<00:08,  4.72it/s]

torch.Size([64, 128, 39, 1])
torch.Size([64, 1, 128, 39])
torch.Size([64, 64, 128, 19])
torch.Size([64, 64, 128, 19])
torch.Size([64, 128, 128, 9])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


...

 98%|█████████████████████████████████████████▉ | 39/40 [00:08<00:00,  5.05it/s]

torch.Size([64, 128, 33, 1])
torch.Size([64, 1, 128, 33])
torch.Size([64, 64, 128, 16])
torch.Size([64, 64, 128, 16])
torch.Size([64, 128, 128, 8])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])
torch.Size([36, 128, 33, 1])
torch.Size([36, 1, 128, 33])
torch.Size([36, 64, 128, 16])
torch.Size([36, 64, 128, 16])
torch.Size([36, 128, 128, 8])
torch.Size([36, 512, 32, 2])
torch.Size([36, 512])
torch.Size([36, 512])
torch.Size([36, 512])
fc통과: torch.Size([36, 507])
torch.Size([36, 512])


100%|███████████████████████████████████████████| 40/40 [00:08<00:00,  4.67it/s]

07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 
07-04 16:51 [MainProcess, 23066] [INFO ]  Iteration:14, loss = 0.557267, acc = 100.000 



/opt/conda/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
def run_challenge_model(model, data, recordings, verbose):
    imputer = model['imputer']
    murmur_classes = model['murmur_classes']
    murmur_classifier = model['murmur_classifier']
    outcome_classes = model['outcome_classes']
    outcome_classifier = model['outcome_classifier']

    # Load features.
    features = get_features(data, recordings)

    # Impute missing data.
    features = features.reshape(1, -1)
    features = imputer.transform(features)

    # Get classifier probabilities.
    murmur_probabilities = murmur_classifier.predict_proba(features)
    murmur_probabilities = np.asarray(murmur_probabilities, dtype=np.float32)[:, 0, 1]
    outcome_probabilities = outcome_classifier.predict_proba(features)
    outcome_probabilities = np.asarray(outcome_probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

In [ ]:
enforce_positives(outputs,)

In [236]:
def enforce_positives(outputs, classes, positive_class):
    num_patients, num_classes = np.shape(outputs)
    j = classes.index(positive_class)

    for i in range(num_patients):
        if np.sum(outputs[i, :]) != 1:
            outputs[i, :] = 0
            outputs[i, j] = 1
    return outputs


In [238]:
outputs

tensor([[0.3853, 0.2572, 0.3575],
        [0.4270, 0.2319, 0.3411],
        [0.3851, 0.2847, 0.3302],
        [0.3279, 0.2757, 0.3964],
        [0.5627, 0.2178, 0.2194],
        [0.3402, 0.3603, 0.2995],
        [0.5247, 0.2314, 0.2438],
        [0.4454, 0.2543, 0.3003],
        [0.4354, 0.2383, 0.3263],
        [0.2537, 0.3609, 0.3854],
        [0.5419, 0.2309, 0.2272],
        [0.2717, 0.4402, 0.2881],
        [0.4025, 0.3599, 0.2376],
        [0.3929, 0.2348, 0.3723],
        [0.2607, 0.2236, 0.5157],
        [0.5612, 0.2177, 0.2211],
        [0.5098, 0.2283, 0.2620],
        [0.5145, 0.2363, 0.2492],
        [0.3438, 0.2432, 0.4130],
        [0.3287, 0.2527, 0.4186],
        [0.2597, 0.2335, 0.5068],
        [0.4649, 0.2315, 0.3035],
        [0.5163, 0.2237, 0.2599],
        [0.3802, 0.3440, 0.2758],
        [0.5036, 0.2404, 0.2560],
        [0.5578, 0.2159, 0.2263],
        [0.5755, 0.2121, 0.2123],
        [0.4593, 0.2539, 0.2867],
        [0.5738, 0.2125, 0.2137],
        [0.540

In [239]:
murmur_classes = ['Present', 'Unknown', 'Absent']

In [ ]:
 # Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
#    outcome_labels = load_outcomes(label_files, outcome_classes)
#    outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)

# For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
#outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
#outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')

In [97]:
import sys
from helper_code import is_integer
#from team_code import train_challenge_model

if __name__ == '__main__':
    # Parse the arguments.
    if not (len(sys.argv) == 3 or len(sys.argv) == 4):
        raise Exception('Include the data and model folders as arguments, e.g., python train_model.py data model.')

    # Define the data and model foldes.
    train_folder = '/Data2/hmd/data_split/murmur/train'
    #test_folder ='/Data2/hmd/data_split/murmur/test'
    model_folder = 'model'

    # Change the level of verbosity; helpful for debugging.
    if len(sys.argv)==4 and is_integer(sys.argv[3]):
        verbose = int(sys.argv[3])
    else:
        verbose = 1

        
        
        
    train_challenge_model(train_folder, model_folder, verbose) ### Teams: Implement this function!!!

Finding data files...
Extracting features and labels from the Challenge data...
Training model...
Training model...


  0%|                                                    | 0/40 [00:00<?, ?it/s]

torch.Size([64, 128, 34, 1])
torch.Size([64, 1, 128, 34])
torch.Size([64, 64, 128, 17])
torch.Size([64, 64, 128, 17])
torch.Size([64, 128, 128, 8])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


  2%|█                                           | 1/40 [00:00<00:08,  4.42it/s]

torch.Size([64, 128, 39, 1])
torch.Size([64, 1, 128, 39])
torch.Size([64, 64, 128, 19])
torch.Size([64, 64, 128, 19])
torch.Size([64, 128, 128, 9])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


...

torch.Size([64, 512])
torch.Size([64, 128, 37, 1])
torch.Size([64, 1, 128, 37])
torch.Size([64, 64, 128, 18])
torch.Size([64, 64, 128, 18])
torch.Size([64, 128, 128, 9])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


 60%|█████████████████████████▊                 | 24/40 [00:05<00:03,  4.89it/s]

torch.Size([64, 128, 47, 1])
torch.Size([64, 1, 128, 47])
torch.Size([64, 64, 128, 23])
torch.Size([64, 64, 128, 23])
torch.Size([64, 128, 128, 11])
torch.Size([64, 512, 32, 2])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
fc통과: torch.Size([64, 507])
torch.Size([64, 512])


 60%|█████████████████████████▊                 | 24/40 [00:05<00:03,  4.15it/s]


KeyboardInterrupt: 

In [109]:
nnet = vgg_hj(num_murmur_classes)
nnet = nnet.cuda()
optimizer = optim.Adam(nnet.parameters(), lr=4e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.91)
criterion = nn.CrossEntropyLoss()

In [110]:
imputer = SimpleImputer().fit(features['hw'])
features['hw'] = imputer.transform(features['hw'])

In [115]:
for iter_ in range(15):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0
    train_epoch_acc = 0.0
    nnet.train()
        
    idx = 0
#    epoch[str(iter_)] = {}
    for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_murmurs):  
            #print(audio_feature.shape)
        
        idx = idx + 1
            #audio_feature = torch.nan_to_num(audio_feature, nan=0)
        audio_feature[audio_feature != audio_feature] = 0
        
            #audio_feature = audio_feature.permute(1,3,0,2)
            #print(audio_feature.shape)
        audio_feature = audio_feature.cuda()
        age = age.permute(1,0).float().cuda()
            #print(age.shape)
        sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
        hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
        preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
        loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
        data_label_torch = data_label_torch.cuda()
                
        data_label_torch = data_label_torch.long()
        outputs = nnet(input=audio_feature, age = age , sex = sex, hw = hw, preg = preg, loc = loc)
        
        loss = criterion(outputs.float(), data_label_torch.squeeze(1))
        train_acc = multi_acc(outputs, data_label_torch.squeeze(1))
            #print(outputs.shape)
            #print(data_label_torch.squeeze(1).shape)
            #print(train_acc)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
            
    logger.info("Iteration:{0}, murmur_loss = {1:.6f}, murmur_acc = {2:.3f} ".format(iter_, running_loss/len(dataloader_train_murmurs), train_epoch_acc/len(dataloader_train_murmurs))


SyntaxError: unexpected EOF while parsing (1011045553.py, line 54)

In [ ]:
save_challenge_model

In [111]:
def save_challenge_model(model_folder, model, classes, m_name) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename = os.path.join(model_folder, m_name + '_model.hdf5')
#    model.save(filename)
    torch.save(model.state_dict(), filename)
    d = {'model': m_name, 'classes': classes,'model_fnm': filename}    
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)

In [ ]:
nnet = vgg_hj(num_outcome_classes)
nnet = nnet.cuda()
nnet.train()

for iter_ in range(15):  # args.end_iter
    start_time = time.time()
    running_loss = 0.0
    train_epoch_acc = 0.0
    nnet.train()
        
    for audio_feature, age, sex, hw, preg, loc, data_label_torch in tqdm(dataloader_train_outcoms):  
            #print(audio_feature.shape)
        
            idx = idx + 1
            #audio_feature = torch.nan_to_num(audio_feature, nan=0)
            audio_feature[audio_feature != audio_feature] = 0
        
            audio_feature = audio_feature.permute(1,3,0,2)
            #print(audio_feature.shape)
            audio_feature = audio_feature.cuda()
            age = age.permute(1,0).float().cuda()
            #print(age.shape)
            sex = sex.permute(1,0).float().cuda()
            #print(sex.shape)
            hw = hw.permute(1,0).float().cuda()
            #print(hw.shape)
            preg = preg.permute(1,0).float().cuda()
            #print(preg.shape)
            loc = loc.permute(1,0).float().cuda()
            #print(loc.shape)
    #        for idx in range(len(data_label_torch)) :
    #            data_label_torch[idx][0] = mapping[int(data_label_torch[idx][0])]
            data_label_torch = data_label_torch.cuda()
                
            data_label_torch = data_label_torch.long()
            outputs = nnet(input=audio_feature, age = age , sex = sex, hw = hw, preg = preg, loc = loc)
        
            loss = criterion(outputs.float(), data_label_torch.squeeze(1))
            train_acc = multi_acc(outputs, data_label_torch.squeeze(1))
            #print(outputs.shape)
            #print(data_label_torch.squeeze(1).shape)
            #print(train_acc)
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_epoch_acc += train_acc.item()
            #if idx % 100 == 0 :
            #    print("lr: ", optimizer.param_groups[0]['lr'])
            #    print('batch : ',idx, ' done')
            
        logger.info("Iteration:{0}, outcome_loss = {1:.6f}, outcome_loss = {2:.3f} ".format(iter_, running_loss/len(dataloader_train_outcoms), train_epoch_acc/len(dataloader_train_outcoms)))
    
    
    # Save the model.
    save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier)

    if verbose >= 1:
        print('Done.')


In [143]:
def save_challenge_model(model_folder, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier):
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename = os.path.join(model_folder, m_name + '_model.hdf5')
    torch.save(model.state_dict(), filename)
    d = {'murmur_classes': murmur_classes, 'murmur_classifier': murmur_classifier, 'outcome_classes': outcome_classes, 'outcome_classifier': outcome_classifier}
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)

In [ ]:
python run_model.py model test_data test_outputs